<a href="https://colab.research.google.com/github/Pradyumna-yes/Project_X/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install requests

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!/usr/bin/env python3
import requests
import csv
from datetime import datetime
import pytz

# Replace 'your_api_key_here' with your actual JCDecaux API key.
API_KEY = 'e42659b6b45ae7dd9ca5cfc3674528e84a28e254'

def fetch_data(api_key):
    api_url = "https://api.jcdecaux.com/vls/v1/stations"
    contract = "dublin"

    full_url = f"{api_url}?contract={contract}&apiKey={api_key}"

    try:
        response = requests.get(full_url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def convert_epoch_to_readable(epoch_time):
    # Assuming the epoch time is in milliseconds
    return datetime.fromtimestamp(epoch_time / 1000, pytz.timezone('Europe/Dublin')).strftime('%Y-%m-%d %H:%M:%S')

def save_to_csv(data, filename):
    if data:
        field_names = ["number", "name", "address", "position", "banking", "bonus", "status", "contract_name", "bike_stands", "available_bike_stands", "available_bikes", "last_update"]

        try:
            with open(filename, mode='w', newline='', encoding='utf-8') as file:
                writer = csv.DictWriter(file, fieldnames=field_names)
                writer.writeheader()
                for station in data:
                    # Convert the last_update field to a readable format
                    station['last_update'] = convert_epoch_to_readable(station['last_update'])
                    writer.writerow(station)
            print(f"Data saved to {filename} successfully!")
        except Exception as e:
            print(f"Error occurred while writing to CSV: {e}")
    else:
        print("No data to save.")

def main():
    data = fetch_data(API_KEY)
    save_to_csv(data, "stations_data.csv")

if __name__ == "__main__":
    main()


Data saved to stations_data.csv successfully!


Test SQl

In [ ]:
pip install pymysql

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.0 MB/s eta 0:00:00


In [ ]:
import pymysql

# Database connection details
host = '140.238.244.101'
user = 'root'
password = '1lnli9uELkeWC7GfUQF01an3Yevq7wXbpkM7vskAyMI652tRhUnin4e8gD4DUOjy'
db = 'Project_X'

# Initialize connection to None
connection = None

try:
    # Attempt to establish a connection
    connection = pymysql.connect(host=host, user=user, password=password, db=db)

    # Create a cursor object using the cursor() method
    cursor = connection.cursor()

    # Execute SQL query using execute() method.
    cursor.execute("SELECT VERSION()")

    # Fetch a single row using fetchone() method.
    data = cursor.fetchone()
    print(f"Database version : {data}")

    # Success message
    print("Connected successfully to the database.")

except pymysql.MySQLError as e:
    print(f"Error connecting to the MySQL database: {e}")

finally:
    # Ensure that the connection is closed even if an error occurs
    if connection:
        connection.close()


Database version : ('8.3.0',)
Connected successfully to the database.


In [ ]:
pip install mysql

In [ ]:
#!/usr/bin/env python3
import requests
import mysql.connector
from datetime import datetime
import pytz

API_KEY = 'e42659b6b45ae7dd9ca5cfc3674528e84a28e254'

def fetch_data(api_key):
    api_url = "https://api.jcdecaux.com/vls/v1/stations"
    contract = "dublin"
    full_url = f"{api_url}?contract={contract}&apiKey={api_key}"

    try:
        response = requests.get(full_url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def connect_to_database(host, database, user, password):
    try:
        conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        return conn
    except mysql.connector.Error as e:
        print(f"Error connecting to MySQL Database: {e}")
        return None

def insert_data_to_db(conn, data):
    if conn is not None and data is not None:
        cursor = conn.cursor()
        insert_query = """
        INSERT INTO stations (number, name, address, position_lat, position_lng, banking, bonus, status, contract_name, bike_stands, available_bike_stands, available_bikes, last_update)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """

        for station in data:
            last_update = datetime.fromtimestamp(station['last_update'] / 1000, pytz.timezone('Europe/Dublin'))
            values = (
                station['number'], station['name'], station['address'],
                station['position']['lat'], station['position']['lng'],
                station['banking'], station['bonus'], station['status'],
                station['contract_name'], station['bike_stands'],
                station['available_bike_stands'], station['available_bikes'],
                last_update
            )
            try:
                cursor.execute(insert_query, values)
            except mysql.connector.Error as e:
                print(f"Error inserting data: {e}")

        conn.commit()
        cursor.close()
        print("Data inserted successfully.")
    else:
        print("No data to insert or database connection is not established.")

def main():
    data = fetch_data(API_KEY)
    conn = connect_to_database('140.238.244.101', 'Project_X', 'root', '1lnli9uELkeWC7GfUQF01an3Yevq7wXbpkM7vskAyMI652tRhUnin4e8gD4DUOjy')
    insert_data_to_db(conn, data)
    if conn is not None:
        conn.close()

if _name_ == "_main_":
  main()


ModuleNotFoundError: No module named 'mysql'

# New Section